# Inspecteer Excel-bestanden in raw_batch_geoloket
Deze notebook toont de eerste 5 regels van elke tabblad in elk bestand in de map `raw_batch_geoloket`.

In [4]:
import os
from pathlib import Path
from openpyxl import load_workbook

# Use correct relative path from the notebooks folder
folder = Path('../input_files/raw_batch_geoloket').resolve()
if not folder.exists():
    raise FileNotFoundError(f"Map niet gevonden: {folder}")

files = [f for f in os.listdir(folder) if f.endswith('.xlsx')]
for fname in files:
    print(f'Bestand: {fname}')
    wb = load_workbook(folder / fname, data_only=True, read_only=True)
    for sheet in wb.sheetnames:
        print(f'  Tabblad: {sheet}')
        ws = wb[sheet]
        for i, row in enumerate(ws.iter_rows(values_only=True), 1):
            print(row)
            if i >= 5:
                break
        print()

Bestand: 87097-1.xlsx
  Tabblad: 87097-1 HB178PB01
('87097-1 HB178PB01', 'MSLV7_lb0845', None, None)
('Timestamp', 'Water Absolute Druk (Pa)', 'Water Temperatuur (ºC)', 'Waterniveau (m NAP)')
(datetime.datetime(2024, 10, 21, 10, 0), 140500.0, None, -4.348)
(datetime.datetime(2024, 10, 21, 11, 0), 140600.0, None, -4.349)
(datetime.datetime(2024, 10, 21, 12, 0), 140640.0, None, -4.347)

  Tabblad: 87097-1 HB179PB01
('87097-1 HB179PB01', 'MSLV7_lb0850', None, None)
('Timestamp', 'Water Absolute Druk (Pa)', 'Water Temperatuur (ºC)', 'Waterniveau (m NAP)')
(datetime.datetime(2024, 10, 18, 16, 0), 119030.0, None, -2.6)
(datetime.datetime(2024, 10, 18, 17, 0), 119090.0, None, -2.593)
(datetime.datetime(2024, 10, 18, 18, 0), 119050.0, None, -2.593)

  Tabblad: 87097-1 HB180PB01
('87097-1 HB180PB01', 'MSLV7_lb0820', None, None)
('Timestamp', 'Water Absolute Druk (Pa)', 'Water Temperatuur (ºC)', 'Waterniveau (m NAP)')
(datetime.datetime(2024, 10, 18, 16, 0), 131560.0, None, -1.082)
(datetime.dat

In [8]:
import pandas as pd
import re
from pathlib import Path

# === CONFIG ===
# Folder with input files
input_folder = Path('../input_files/raw_batch_geoloket').resolve()
output_format = 'csv'  # 'csv' of 'xlsx'
output_dir = Path('../output_files').resolve()
output_dir.mkdir(exist_ok=True)

# === SCRIPT ===
all_summaries = []
files = [f for f in input_folder.iterdir() if f.suffix == '.xlsx']
for input_file in files:
    try:
        xls = pd.ExcelFile(input_file)
    except Exception as e:
        print(f'Fout bij openen van bestand {input_file}: {e}')
        continue

    summary = []
    for sheet in xls.sheet_names:
        # Read first two rows: row 0 for name, row 1 for headers
        try:
            preview = pd.read_excel(xls, sheet_name=sheet, nrows=2, header=None)
        except Exception as e:
            print(f'Fout bij lezen van eerste rijen van sheet {sheet} in {input_file.name}: {e}')
            continue
        if preview.empty or len(preview) < 2:
            print(f'{input_file.name} - Sheet {sheet}: niet genoeg rijen voor naam en headers!')
            continue
        out_name = str(preview.iloc[0,0]).strip()  # Use first cell of first row as output name
        # Now read the actual data, using row 2 (index 1) as header
        try:
            df = pd.read_excel(xls, sheet_name=sheet, header=1)
        except Exception as e:
            print(f'Fout bij lezen van sheet {sheet} in {input_file.name}: {e}')
            continue

        # Zoek kolommen
        ts_col = next((col for col in df.columns if 'Timestamp' in col), None)
        wn_col = next((col for col in df.columns if 'Waterniveau (m NAP)' in col), None)
        if not ts_col or not wn_col:
            print(f'{input_file.name} - Sheet {sheet}: vereiste kolommen niet gevonden!')
            continue

        df_out = df[[ts_col, wn_col]].copy()
        df_out = df_out.rename(columns={wn_col: 'Waterniveau'})
        df_out[ts_col] = pd.to_datetime(df_out[ts_col], errors='coerce')

        out_path = output_dir / f'{out_name}.{output_format}'
        if output_format == 'csv':
            df_out.to_csv(out_path, index=False)
        elif output_format == 'xlsx':
            df_out.to_excel(out_path, index=False)
        else:
            print(f'Onbekend output-formaat: {output_format}')
            continue

        summary.append((out_name, len(df_out)))

    if summary:
        print(f'Samenvatting voor {input_file.name}:')
        for name, rows in summary:
            print(f'{name}: {rows} rijen opgeslagen.')
        all_summaries.append((input_file.name, summary))
    else:
        print(f'Geen geschikte sheets gevonden of opgeslagen in {input_file.name}.')

Samenvatting voor 87097-1.xlsx:
87097-1 HB178PB01: 4576 rijen opgeslagen.
87097-1 HB179PB01: 4650 rijen opgeslagen.
87097-1 HB180PB01: 4623 rijen opgeslagen.
87097-1 HB181PB01: 4616 rijen opgeslagen.
87097-1 HB182PB01: 4595 rijen opgeslagen.
87097-1 HB183PB01: 4650 rijen opgeslagen.
87097-1 HB184PB01: 4625 rijen opgeslagen.
87097-1 HB185PB01: 4651 rijen opgeslagen.
87097-1 HB186PB01: 4651 rijen opgeslagen.
87097-1 HB188PB01: 4640 rijen opgeslagen.
87097-1 HB189PB01: 4645 rijen opgeslagen.
87097-1 HB190PB01: 4639 rijen opgeslagen.
87097-1 HB191PB01: 4641 rijen opgeslagen.
